# 4.4 Detector & Descriptor Benchmarking

Now that we have met some of the most interesting keypoint detectors and descriptors, it would be interesting to test them and compare their results in terms of number of **detections, robustness, invariance and performance**. In the context of our photo-stitching application, not all the keypoint detectors and descriptors seem to perform the same, right?.

Thus, in this notebook, you are asked to **evaluate** the following methods:

- Harris + NCC
- Harris + ORB (descriptor)
- ORB 
- SIFT
- SURF

in images that suffer **changes** in:

- lighting conditions
- rotation
- scale
- point of view

So, for each situation, you'll be provided with a pair of images that you will have to use to **detect, describe and match** the above-mentioned keypoints. After that, plot in a bar chart the following statistics:

- average number of keypoints detected in the images,
- number of found matches,
- time spent per keypoint at detection (including description)*, and
- time spent per match during matching*.

*use `time.process_time()` from the [`time`](https://docs.python.org/3/library/time.html) package to measure time.

In [ ]:
# preamble
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
import time
matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)
images_path = './images/'

# for pysift
import sys
sys.path.append("..")
from utils.third_party import pysift #https://github.com/rmislam/PythonSIFT

## Prepare output

In [ ]:
# Create output vectors
# We have 5 methods and 4 different scenarios
stats_kps = np.zeros((4,5))
stats_mat = np.zeros((4,5))
stats_tdet = np.zeros((4,5))
stats_tmat = np.zeros((4,5))

## 4.4.1 Preliminary functions

### **<span style="color:green"><b><i>ASSIGNMENT 1a:  For Harris and NCC (brought from 4.1 and adapted)</i></b></span>**
First, take the code you implemented in notebook 4.1 for the Harris method and convert it in two functions to:
- detect Harris keypoints (return a list of `cv2.Keypoint`).
- match Harris keypoints using NCC (return a list of `cv2.DMatch`). Use in here the `non-max-suppresion` method we provided to you.

In [ ]:
# Define a function to detect Harris
def detectHarris(image,w_size,sobel_size,k):
    # compute Harris
    harris = cv2.cornerHarris(image,w_size,sobel_size,k)
    # perform non-max-suppression
    r,c = nonmaxsuppts(harris, 3, 0.1*harris.max())
    # convert to a keypoint list
    kps = [cv2.KeyPoint(c[i], r[i], 1) for i in range(len(r))]
    
    return kps
    
# ... and another one to match them
def matchHarris(image_l,image_r,kps_l,kps_r):
    # augment border
    w_temp = 20 # window size
    image_l_pad = cv2.copyMakeBorder(image_l, w_temp, w_temp, w_temp, w_temp, cv2.BORDER_REFLECT)
    image_r_pad = cv2.copyMakeBorder(image_r, w_temp, w_temp, w_temp, w_temp, cv2.BORDER_REFLECT)
    
    # initialize matches list
    matches = []
    
    c_r = [int(round(kps_r[i].pt[0])) for i in range(len(kps_r))]
    r_r = [int(round(kps_r[i].pt[1])) for i in range(len(kps_r))]
    
    # for each keypoint in left image
    for p_index in range(len(kps_l)):

        c_l = int(round(kps_l[p_index].pt[0]))
        r_l = int(round(kps_l[p_index].pt[1]))
        
        # get the template (note the w_temp offset because of the padding in previous step)
        p_r, p_c = r_l+w_temp, c_l+w_temp
        template = image_l_pad[p_r-w_temp:p_r+w_temp+1,p_c-w_temp:p_c+w_temp+1]

        # compute NCC of the left keypoint in right image
        ncc = cv2.matchTemplate(image_r_pad, template, cv2.TM_CCORR_NORMED)

        # find max value in NCC (only at the right keypoints)
        max_value = np.amax(ncc[r_r,c_r])

        # if match is good
        if max_value > 0.95:

            # include in match list
            [max_index] = np.where(ncc[r_r,c_r] == max_value)
            matches.append((p_index, max_index))

    # cast matches list to cv2.DMatch list
    matches = np.asarray(matches)
    matches = [cv2.DMatch(matches[i,0], matches[i,1],1) for i in range(matches.shape[0])]
        
    return matches

# This method has been provided to you
from scipy import signal
def nonmaxsuppts(cim, radius, thresh):
    """ Binarize and apply non-maximum suppresion.   
    
        Args:
            cim: the harris 'R' image
            radius: the aperture size of local maxima window
            thresh: the threshold value for binarization
                    
        Returns: 
            r, c: two numpy vectors being the row (r) and the column (c) of each keypoint
    """   
    
    rows, cols = np.shape(cim)
    sze = 2 * radius + 1
    mx = signal.order_filter(cim, np.ones([sze, sze]), sze ** 2 - 1)
    bordermask = np.zeros([rows, cols]);
    bordermask[radius:(rows - radius), radius:(cols - radius)] = 1
    cim = np.array(cim)
    r, c = np.where((cim == mx) & (cim > thresh) & (bordermask == 1))
    return r, c

### **<span style="color:green"><b><i>ASSIGNMENT 1b:  Create a process function for each method</i></b></span>** 

Now, **create a method** for each of the proposed algorithms:
- Harris + NCC
- Harris + ORB descriptor
- ORB
- SIFT
- SURF

that performs all the desired tasks for a pair of input images (insert both the grayscale and the color version of them). These functions should do the following:
- compute keypoints and descriptors from the grayscale image (also store the number of detected keypoint and measure the time spent in the process)
- find matches (also store the number of matches found and measure the time spent in the process)
- plot the resulting matches on the color images.
- return the **average number of keypoints per image**, the **number of matches**, the **detection time** and the **matching time**.

**Once you have this, you just need to call them for each pair of images!**

In [ ]:
def process_Harris_NCC(image_l, image_l_gray, image_r, image_r_gray):
    # parameters:
    w_size = 2
    sobel_size = 3
    k = 0.05

    # compute Harris corners
    st_det = time.process_time() # Start timer
    kps_l = detectHarris(image_l_gray,w_size,sobel_size,k)
    kps_r = detectHarris(image_r_gray,w_size,sobel_size,k) # same for the right ones
    det_time = round(time.process_time()-st_det,5)

    kps = len(kps_l)+len(kps_r)

    # save partial stats
    num_kps = kps*0.5 # average number of keypoints
    tdet = det_time/kps
    
    # match using NCC
    st_mat = time.process_time() # Start timer
    matches = matchHarris(image_l_gray, image_r_gray, kps_l, kps_r)
    mat_time = round(time.process_time()-st_mat,5)

    # save partial stats
    num_matches = len(matches)
    tmat = mat_time/num_matches
    
    # plot results
    this_image = np.copy(image_l)
    this_image = cv2.drawMatches(image_l,kps_l,image_r,kps_r,matches,this_image,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.imshow(this_image)
    
    return num_kps, num_matches, tdet, tmat

In [ ]:
def process_Harris_ORB(image_l, image_l_gray, image_r, image_r_gray):
    # parameters:
    w_size = 2
    sobel_size = 3
    k = 0.05

    # compute Harris corners
    st_det = time.process_time() # Start timer
    kps_l = detectHarris(image_l_gray,w_size,sobel_size,k)
    kps_r = detectHarris(image_r_gray,w_size,sobel_size,k) # same for the right ones
    det_time = round(time.process_time()-st_det,5)

    kps = len(kps_l)+len(kps_r)

    # save partial stats
    num_kps = kps*0.5 # average number of keypoints
    tdet = det_time/kps
    
    # compute descriptors
    orb = cv2.ORB_create()

    st_det_desc = time.process_time() # Start timer
    kps_l, des_l = orb.compute(image_l_gray, kps_l)
    kps_r, des_r = orb.compute(image_r_gray, kps_r)
    det_desc_time = round(time.process_time()-st_det_desc,5)

    # update partial stats
    tdet = tdet + det_desc_time/num_kps # add descriptor computation to detection time
    
    # Match descriptors.
    st_mat = time.process_time() # Start timer
    matches = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True).match(des_l,des_r)
    mat_time = round(time.process_time()-st_mat,5)

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    
    # save partial stats
    num_matches = len(matches)
    tmat = mat_time/num_matches

    # plot results
    this_image = np.copy(image_l)
    this_image = cv2.drawMatches(image_l,kps_l,image_r,kps_r,matches,this_image,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.imshow(this_image)
    
    return num_kps, num_matches, tdet, tmat

In [ ]:
def process_ORB(image_l, image_l_gray, image_r, image_r_gray):
    # -- create the ORB detector
    orb = cv2.ORB_create()

    # -- detect ORB keypoints
    st_det = time.process_time() # Start timer
    kps_l = orb.detect(image_l_gray,None)
    kps_r = orb.detect(image_r_gray,None)

    # -- compute the descriptors with ORB
    kps_l, des_l = orb.compute(image_l_gray, kps_l)
    kps_r, des_r = orb.compute(image_r_gray, kps_r)
    det_time = round(time.process_time()-st_det,5)
    
    kps = len(kps_l)+len(kps_r)
    
    # save partial stats
    num_kps = kps*0.5 # average number of keypoints
    tdet = det_time/kps

    # Match descriptors.
    st_mat = time.process_time() # Start timer
    matches = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True).match(des_l,des_r)
    mat_time = round(time.process_time()-st_mat,5)

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    
    # save partial stats
    num_matches = len(matches)
    tmat = mat_time/num_matches

    # Display both images side-by-side along with the matches
    this_image = np.copy(image_l)
    this_image = cv2.drawMatches(image_l,kps_l,image_r,kps_r,matches,this_image,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.imshow(this_image)
    
    return num_kps, num_matches, tdet, tmat

In [ ]:
def process_SIFT(image_l, image_l_gray, image_r, image_r_gray):
    
    # OpenCV
#     sift = cv2.xfeatures2d.SIFT_create(300)
#     st_det = time.process_time() # Start timer
#     kps_l, des_l = sift.detectAndCompute(image_l_gray,None)
#     kps_r, des_r = sift.detectAndCompute(image_r_gray,None)
#     det_time = round(time.process_time()-st_det,5)
    
    # pysift
    st_det = time.process_time() # Start timer
    kps_l, des_l = pysift.computeKeypointsAndDescriptors(image_l_gray)
    kps_r, des_r = pysift.computeKeypointsAndDescriptors(image_r_gray)
    det_time = round(time.process_time()-st_det,5)
    
    kps = len(kps_l)+len(kps_r)
    
    # save partial stats
    num_kps = kps*0.5 # average number of keypoints
    tdet = det_time/kps

    # Call knnMatch
    st_mat = time.process_time() # Start timer
    matches = cv2.BFMatcher().knnMatch(des_l,des_r, k=2)
    good = []
    # For each match
    for m,n in matches:
        # If first two distances are not close
        if m.distance < 0.75*n.distance:
            # It is a good match! Add it to the list
            good.append(m)
    mat_time = round(time.process_time()-st_mat,5)
    
    # save partial stats
    num_matches = len(good)
    tmat = mat_time/num_matches

    # Display both images side-by-side along with the matches
    this_image = np.copy(image_l)
    this_image = cv2.drawMatches(image_l,kps_l,image_r,kps_r,good,this_image, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imshow(this_image)
    
    return num_kps, num_matches, tdet, tmat

In [ ]:
def process_SURF(image_l, image_l_gray, image_r, image_r_gray):
    
    surf = cv2.xfeatures2d.SURF_create(2000)
    
    st_det = time.process_time() # Start timer
    kps_l, des_l = surf.detectAndCompute(image_l_gray,None)
    kps_r, des_r = surf.detectAndCompute(image_r_gray,None)
    det_time = round(time.process_time()-st_det,5)
    
    kps = len(kps_l)+len(kps_r)
    
    # save partial stats
    num_kps = kps*0.5 # average number of keypoints
    tdet = det_time/kps
    
    # Call knnMatch
    st_mat = time.process_time() # Start timer
    matches = cv2.BFMatcher().knnMatch(des_l,des_r, k=2)
    good = []
    # For each match
    for m,n in matches:
        # If first two distances are not close
        if m.distance < 0.60*n.distance:
            # It is a good match! Add it to the list
            good.append(m)
    mat_time = round(time.process_time()-st_mat,5)
    
    # save partial stats
    num_matches = len(good)
    tmat = mat_time/num_matches

    # Display both images side-by-side along with the matches
    this_image = np.copy(image_l)
    this_image = cv2.drawMatches(image_l,kps_l,image_r,kps_r,good,this_image, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imshow(this_image)
    
    return num_kps, num_matches, tdet, tmat    

## 4.4.2 Testing the methods!

### **<span style="color:green"><b><i>ASSIGNMENT 2a: Changes in lighting conditions</i></b></span>** 

Use `bright1.png` and `bright2.png` images, which contain the same scene but with different lighting conditions.

<img src="./images/bright1.png" width="300" align="left"/><img src="./images/bright2.png" width="300" align="rigth"/>

#### Read the images and convert them to gray (they will be used for all the methods)

In [ ]:
# Write your code here!
## Read images and convert to gray
image_l = cv2.imread(images_path + 'bright1.png')
image_r = cv2.imread(images_path + 'bright2.png')
image_l = cv2.cvtColor(image_l,cv2.COLOR_BGR2RGB)
image_r = cv2.cvtColor(image_r,cv2.COLOR_BGR2RGB)
image_l_gray = cv2.cvtColor(image_l,cv2.COLOR_RGB2GRAY)
image_r_gray = cv2.cvtColor(image_r,cv2.COLOR_RGB2GRAY)

#### Make the tests

In [ ]:
# HARRIS + NCC
stats_kps[0,0],stats_mat[0,0],stats_tdet[0,0],stats_tmat[0,0] = process_Harris_NCC(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# HARRIS + ORB
stats_kps[0,1],stats_mat[0,1],stats_tdet[0,1],stats_tmat[0,1] = process_Harris_ORB(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# ORB
stats_kps[0,2],stats_mat[0,2],stats_tdet[0,2],stats_tmat[0,2] = process_ORB(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# SIFT
stats_kps[0,3],stats_mat[0,3],stats_tdet[0,3],stats_tmat[0,3] = process_SIFT(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# SURF
stats_kps[0,4],stats_mat[0,4],stats_tdet[0,4],stats_tmat[0,4] = process_SURF(image_l, image_l_gray, image_r, image_r_gray)

### **<span style="color:green"><b><i>ASSIGNMENT 2b: Changes in rotation</i></b></span>** 

Use `rotate1.png` and `rotate2.png` images.

<img src="./images/rotate1.png" width="300" align="left"/><img src="./images/rotate2.png" width="300" align="rigth"/>

#### Read the images and convert them to gray (they will be used for all the methods)

In [ ]:
# Write your code here!
## Read images and convert to gray
image_l = cv2.imread(images_path + 'rotate1.png')
image_r = cv2.imread(images_path + 'rotate2.png')
image_l = cv2.cvtColor(image_l,cv2.COLOR_BGR2RGB)
image_r = cv2.cvtColor(image_r,cv2.COLOR_BGR2RGB)
image_l_gray = cv2.cvtColor(image_l,cv2.COLOR_RGB2GRAY)
image_r_gray = cv2.cvtColor(image_r,cv2.COLOR_RGB2GRAY)

#### Make the tests

In [ ]:
# HARRIS + NCC
stats_kps[1,0],stats_mat[1,0],stats_tdet[1,0],stats_tmat[1,0] = process_Harris_NCC(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# HARRIS + ORB
stats_kps[1,1],stats_mat[1,1],stats_tdet[1,1],stats_tmat[1,1] = process_Harris_ORB(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# ORB
stats_kps[1,2],stats_mat[1,2],stats_tdet[1,2],stats_tmat[1,2] = process_ORB(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# SIFT
stats_kps[1,3],stats_mat[1,3],stats_tdet[1,3],stats_tmat[1,3] = process_SIFT(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# SURF
stats_kps[1,4],stats_mat[1,4],stats_tdet[1,4],stats_tmat[1,4] = process_SURF(image_l, image_l_gray, image_r, image_r_gray)

### **<span style="color:green"><b><i>ASSIGNMENT 2c: Changes in scale</i></b></span>** 

Use `scale1.png` and `scale2.png` images.

<img src="./images/scale1.png" width="300" align="left"/><img src="./images/scale2.png" width="300" align="rigth"/>

#### Read the images and convert them to gray (they will be used for all the methods)

In [ ]:
# Write your code here!
## Read images and convert to gray
image_l = cv2.imread(images_path + 'scale1.png')
image_r = cv2.imread(images_path + 'scale2.png')
image_l = cv2.cvtColor(image_l,cv2.COLOR_BGR2RGB)
image_r = cv2.cvtColor(image_r,cv2.COLOR_BGR2RGB)
image_l_gray = cv2.cvtColor(image_l,cv2.COLOR_RGB2GRAY)
image_r_gray = cv2.cvtColor(image_r,cv2.COLOR_RGB2GRAY)

#### Make the tests

In [ ]:
# HARRIS + NCC
stats_kps[2,0],stats_mat[2,0],stats_tdet[2,0],stats_tmat[2,0] = process_Harris_NCC(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# HARRIS + ORB
stats_kps[2,1],stats_mat[2,1],stats_tdet[2,1],stats_tmat[2,1] = process_Harris_ORB(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# ORB
stats_kps[2,2],stats_mat[2,2],stats_tdet[2,2],stats_tmat[2,2] = process_ORB(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# SIFT
stats_kps[2,3],stats_mat[2,3],stats_tdet[2,3],stats_tmat[2,3] = process_SIFT(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# SURF
stats_kps[2,4],stats_mat[2,4],stats_tdet[2,4],stats_tmat[2,4] = process_SURF(image_l, image_l_gray, image_r, image_r_gray)

### **<span style="color:green"><b><i>ASSIGNMENT 2d: Changes in point of view</i></b></span>** 

Use `pov1.png` and `pov2.png` images.

<img src="./images/pov1.png" width="300" align="left"/><img src="./images/pov2.png" width="300" align="rigth"/>

#### Read the images and convert them to gray (they will be used for all the methods)

In [ ]:
# Write your code here!
## Read images and convert to gray
image_l = cv2.imread(images_path + 'pov1.png')
image_r = cv2.imread(images_path + 'pov2.png')
image_l = cv2.cvtColor(image_l,cv2.COLOR_BGR2RGB)
image_r = cv2.cvtColor(image_r,cv2.COLOR_BGR2RGB)
image_l_gray = cv2.cvtColor(image_l,cv2.COLOR_RGB2GRAY)
image_r_gray = cv2.cvtColor(image_r,cv2.COLOR_RGB2GRAY)

#### Make the tests

In [ ]:
# HARRIS + NCC
stats_kps[3,0],stats_mat[3,0],stats_tdet[3,0],stats_tmat[3,0] = process_Harris_NCC(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# HARRIS + ORB
stats_kps[3,1],stats_mat[3,1],stats_tdet[3,1],stats_tmat[3,1] = process_Harris_ORB(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# ORB
stats_kps[3,2],stats_mat[3,2],stats_tdet[3,2],stats_tmat[3,2] = process_ORB(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# SIFT
stats_kps[3,3],stats_mat[3,3],stats_tdet[3,3],stats_tmat[3,3] = process_SIFT(image_l, image_l_gray, image_r, image_r_gray)

In [ ]:
# SURF
stats_kps[3,4],stats_mat[3,4],stats_tdet[3,4],stats_tmat[3,4] = process_SURF(image_l, image_l_gray, image_r, image_r_gray)

## 4.4.3 Visually analyzing the results

### **<span style="color:green"><b><i>ASSIGNMENT 3: Plot the results</i></b></span>** 

Now generate plots with the following metrics:
- average number of keypoints detected in the images
- number of found matches
- time spent per keypoint at detection (including description)
- time spent per match during matching

Create a 4x4 plot with [bar](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.bar.html) graphs with a **row for each metric** (*#keypoints*, *#matches*, *detection time*, and *matching time*) and a **column for each test** (*lighting conditions*, *rotation*, *scale* and *point-of-view*).

In [ ]:
# Generate graphs
matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)

titles = ('Lighting', 'Rotation', 'Scale', 'Point-of-view')
objects = ('H+NCC', 'H+ORB', 'ORB', 'SIFT', 'SURF')
y_pos = np.arange(len(objects))

# keypoints
for i in range(1,5):
    plt.subplot(4,4,i)
    plt.bar(y_pos, stats_kps[i-1,:], align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('# kps')
    plt.title(titles[i-1])

# matches
for i in range(1,5):
    plt.subplot(4,4,i+4)
    plt.bar(y_pos, stats_mat[i-1,:], align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('# matches')
    plt.title(titles[i-1])

# time per detection
for i in range(1,5):
    ax = plt.subplot(4,4,i+8)
    ax.set_yscale('log')
    plt.bar(y_pos, stats_tdet[i-1,:], align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('t_det [s]')
    plt.title(titles[i-1])

# time per match
for i in range(1,5):
    ax = plt.subplot(4,4,i+12)
    ax.set_yscale('log')
    plt.bar(y_pos, stats_tmat[i-1,:], align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('t_mat [s]')
    plt.title(titles[i-1])

plt.show()

## Conclusion

Now you have finished these tests, answer the following questions:

- Are the evaluated methods invariant to these changes? <br />
  
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- Which one would you use if you should work with each kind of images? <br />
    
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
  
- Which one would you use if you need a real-time system? <br />

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
  
- If there is any method NOT invariant against a certain change, can you think in any solution to make it more robust against that? <br />

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>